# ETL Pipeline - Uber Dataset

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import logging
import time
import os

# Configuração básica de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [2]:
def connect_to_postgres():
    """Tenta conectar ao banco de dados com múltiplas tentativas."""
    db_user = os.getenv('POSTGRES_USER', 'admin')
    db_password = os.getenv('POSTGRES_PASSWORD', 'admin')
    db_name = os.getenv('POSTGRES_DB', 'postgres')
    db_host = 'postgres' # Nome do serviço do banco de dados no docker-compose
    
    conn_string = f"postgresql://{db_user}:{db_password}@{db_host}/{db_name}"
    
    retries = 5
    while retries > 0:
        try:
            logging.info("Tentando conectar ao PostgreSQL...")
            engine = create_engine(conn_string)
            connection = engine.connect()
            logging.info("Conexão com o PostgreSQL estabelecida com sucesso!")
            return engine, connection
        except Exception as e:
            logging.error(f"Falha ao conectar: {e}")
            retries -= 1
            logging.info(f"Tentando novamente em 5 segundos... ({retries} tentativas restantes)")
            time.sleep(5)
    
    logging.critical("Não foi possível conectar ao banco de dados após várias tentativas.")
    return None, None

## Etapa 1 - Extract

In [3]:
engine, connection = connect_to_postgres()

if not engine:
    raise RuntimeError("Não foi possível conectar ao banco de dados. Abortando o ETL.")

# --- Etapa 1: Extract --
logging.info("Iniciando Etapa 1: Extract")
raw_path = '/raw/uber-dataset.csv'
df_raw = pd.read_csv(raw_path)
logging.info(f"{len(df_raw)} linhas lidas do arquivo CSV.")

# --- Etapa 2: Transform (Super Simplificada para Teste) ---
logging.info("Iniciando Etapa 2: Transform")
df = df_raw[['Booking Status']].copy()

# 2. Renomeia a coluna para o nome desejado no banco de dados.
df.rename(columns={'Booking Status': 'bookingStatus'}, inplace=True)

logging.info("Transformação de teste concluída. DataFrame final:")
print(df.head()) # Mostra as primeiras linhas do DataFrame final

# --- Etapa 3: Load ---
logging.info("Iniciando Etapa 3: Load")
try:
    table_name = 'uber_ride_bookings'
    df.to_sql(table_name, engine, if_exists='replace', index=False)
    logging.info(f"Dados de teste carregados com sucesso na tabela '{table_name}'!")
except Exception as e:
    logging.error(f"Erro ao carregar dados no banco: {e}")
finally:
    connection.close()
    logging.info("Conexão com o banco de dados fechada.")

2025-10-09 00:47:37,538 - INFO - Tentando conectar ao PostgreSQL...


2025-10-09 00:47:37,597 - INFO - Conexão com o PostgreSQL estabelecida com sucesso!


2025-10-09 00:47:37,598 - INFO - Iniciando Etapa 1: Extract


2025-10-09 00:47:38,246 - INFO - 150000 linhas lidas do arquivo CSV.


2025-10-09 00:47:38,247 - INFO - Iniciando Etapa 2: Transform


2025-10-09 00:47:38,250 - INFO - Transformação de teste concluída. DataFrame final:


2025-10-09 00:47:38,254 - INFO - Iniciando Etapa 3: Load


     bookingStatus
0  No Driver Found
1       Incomplete
2        Completed
3        Completed
4        Completed


2025-10-09 00:47:39,469 - INFO - Dados de teste carregados com sucesso na tabela 'uber_ride_bookings'!


2025-10-09 00:47:39,470 - INFO - Conexão com o banco de dados fechada.
